In [1]:
import pyspark
import os
import warcio
import re
import nltk
nltk.download('words')
from nltk.corpus import words
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as sql_sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.clustering import LDA
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, explode, lit,array
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from scipy.sparse import csr_matrix
spark = SparkSession.builder \
    .appName("myApp") \
    .config("spark.driver.memory", "32g") \
    .getOrCreate()
spark

[nltk_data] Error loading words: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     Hostname mismatch, certificate is not valid for
[nltk_data]     'raw.githubusercontent.com'. (_ssl.c:1131)>
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/19 08:22:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/19 08:22:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/19 08:22:59 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/05/19 08:22:59 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/05/19 08:22:59 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/05/19 08:22:59 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/05/19 08:22:59 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
23/05/19 08:22:59 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.


#### Defining a schema and the corresponding data types to store the web crawl data.
### Attributions :
##### https://groups.google.com/g/common-crawl/c/yEP1Lt1_B4E

In [2]:
schema = StructType([
    StructField("WARC-Type", StringType(), True),
    StructField("WARC-Target-URI", StringType(), True),
    StructField("WARC-Date", StringType(), True),
    StructField("WARC-Record-ID", StringType(), True),
    StructField("WARC-Refers-To", StringType(), True),
    StructField("WARC-Block-Digest", StringType(), True),
    StructField("WARC-Identified-Content-Language", StringType(), True),
    StructField("Content-Type", StringType(), True),
    StructField("Content-Length", IntegerType(), True),
    StructField("Content", StringType(), True)
])

#### Reading the WARC data from the file .wet file, and storing the headers and content in a list 'records'.
### Attributions :
##### https://www.programcreek.com/python/example/127575/warcio.archiveiterator.ArchiveIterator

In [3]:
records = []
with open('crawl.wet.gz', 'rb') as data:
    for i in warcio.archiveiterator.ArchiveIterator(data):
        headers = dict(i.rec_headers.headers)
        headers['Content-Length'] = int(headers['Content-Length'])
        headers['Content'] = i.raw_stream.read().decode('utf-8')
        records.append(headers)

#### Creating a spark dataframe out of the list records in the format of the schema built above

In [4]:
spark_df = spark.createDataFrame(records, schema=schema)

#### Tokenizing the 'Content' column in the DataFrame using a regex pattern that matches non-word characters, and storing the tokens in a new column 'words'.
#### Attributions :
##### https://www.sparkitecture.io/natural-language-processing/data-preparation

In [5]:
tokenizer = RegexTokenizer(inputCol="Content", outputCol="words", pattern="\\W")
tokenized_df = tokenizer.transform(spark_df)

#### Removing stopwords from the 'words' column, and storing the filtered words in a new column 'filtered'.
#### Attributions :
##### https://ashokpalivela.medium.com/multi-class-text-classification-using-spark-ml-in-python-b8d2a6545cb

In [6]:
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered")
filtered_df = stopwords_remover.transform(tokenized_df)

#### Instantiated a CountVectorizer with the specified parameters to vectorize the 'english_words' column, and stored the results in a new column 'rawFeatures'.  Additionally, calculated the TF-IDF values of the words in the 'rawFeatures' column, and stored the results in a new column 'features'.

In [7]:
english_words = set(words.words())
def english(words):
    return [word for word in words if word.lower() in english_words]
english_udf = udf(english, ArrayType(StringType()))
english_df = filtered_df.withColumn("english_words", english_udf("filtered"))

In [ ]:
vectorizer = CountVectorizer(inputCol="english_words", outputCol="rawFeatures", vocabSize=5000, minDF=5.0)
vectorized_df = vectorizer.fit(english_df).transform(english_df)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf_model = idf.fit(vectorized_df)
tfidf_vectors = idf_model.transform(vectorized_df)

23/05/19 08:23:14 WARN TaskSetManager: Stage 0 contains a task of very large size (41255 KiB). The maximum recommended task size is 1000 KiB.


23/05/19 08:24:21 WARN TaskSetManager: Stage 4 contains a task of very large size (41255 KiB). The maximum recommended task size is 1000 KiB.


#### Training an LDA model on the 'features' column, with the specified number of topics and iterations and extracting the top terms of each topicfrom the LDA model, and store them in a new DataFrame 'topics_with_terms'.
### Attributions :
##### https://medium.com/analytics-vidhya/distributed-topic-modelling-using-spark-nlp-and-spark-mllib-lda-6db3f06a4da3

In [ ]:
num_topics = 20
lda = LDA(k=num_topics, maxIter=10)
lda_model = lda.fit(tfidf_vectors)

#### extracting the vocabulary that the model has learned

In [ ]:
vectorizer_model = vectorizer.fit(english_df)
vocab = vectorizer_model.vocabulary

In [ ]:
topics = lda_model.describeTopics(maxTermsPerTopic=10)
topic_terms_udf = udf(lambda indices: [vocab[i] for i in indices], ArrayType(StringType()))
topics_with_terms = topics.withColumn("topic_terms", topic_terms_udf("termIndices"))
topics_with_terms.select("topic", "topic_terms").show(truncate=False)

In [ ]:
topics_with_terms.toPandas()

In [ ]:
n = 3
top_topics = topics_with_terms.select("topic", "topic_terms").limit(n).collect()
top_topics_list = [(row.topic, row.topic_terms) for row in top_topics]

In [ ]:
print("The top", n, "topics are:", top_topics_list)

#### Defining a function to convert a sparse vector from the 'rawFeatures' column to a SciPy sparse matrix. I have converted each vector in the 'rawFeatures' column to a SciPy sparse matrix, and stored them in a list.
### Attributions :
##### https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

In [ ]:
def scipy_sparse(vector):
    return csr_matrix((vector.values, vector.indices, [0, len(vector.values)]), shape=(1, len(vector)))
sparse_matrices = [scipy_sparse(vector) for vector in vectorized_df.select("rawFeatures").rdd.map(lambda x: x[0]).collect()]

#### Converting scipy sparse matrices to a gensim corpus

In [ ]:
def gensim_corpus(sparse_matrices):
    for matrix in sparse_matrices:
        yield [(i, float(matrix[0, i])) for i in matrix.indices]
corpus = list(gensim_corpus(sparse_matrices))

#### Creating a Gensim Dictionary from the vocabulary of the CountVectorizer and filtering out the outliers.
### Attributions:
##### https://www.tutorialspoint.com/gensim/gensim_creating_a_dictionary.htm
##### https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.filter_extremes.html

In [ ]:
gensim_vocab = gensim.corpora.Dictionary([vectorizer_model.vocabulary])
gensim_vocab.filter_extremes(no_below=1, no_above=1.0, keep_n=None)

#### Training a Gensim LDA model on the Gensim corpus with the specified number of topics and preparing the Gensim LDA model's data for visualization using the pyLDAvis library.
### Attributions :
##### https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
gensim_lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=gensim_vocab)

In [ ]:
vis = gensimvis.prepare(gensim_lda_model, corpus, gensim_vocab)

In [ ]:
pyLDAvis.display(vis)